##### import
- 수정대상 = 분납상태 : 진행 & 현재결과 : 항고 & 분납메모 : 미결정 or ""
- 수정대상건에 대해 전산수정작업 마친 후 t+1일에 다운로드

In [36]:
import os, re, pickle
import pandas as pd
import numpy as np
from os.path import join
from tqdm import tqdm
import warnings
warnings.simplefilter("ignore")

# 파일 읽기
path_base = r"D:\3.자산\전산 dataset"
########################################################
company = "솔림"
workdate = "240131" # 폴더명
name_dataset = "개인회생새창_20240201_1035"
# previous_workdate = "240108" # 폴더명
# previous_name_dataset = "개인회생새창_20240109_0943_기준데이터" # 기준데이터 계좌키 미분리
########################################################
path_dataset = join(path_base, company, workdate)
# path_previous = join(path_base, company, previous_workdate, previous_name_dataset+".xlsx") # 전체시트 읽어야 하므로 pkl 아님
path_result = join(path_dataset, name_dataset+"_기준데이터.xlsx")

rehabilitation_dtype = {'채무자키':str, '계좌키':str, '분납키':str, '사건키':str}

# 금지명령(중지명령) : 전산은 금지명령이 기각되는 경우를 무시하고 신청서제출만으로 금지로 처리함.
# 접수 : 금지명령신청만 한 경우나, 기각된 경우를 따로 표시하기 위해 추가함
항고인용사건_현재결과종류 = {"개시":"개인회생절차개시결정\x7f", "금지": r"금지명령\(|중지명령\x7f", # r"(금지|중지)명령신청서? 제출\x7f", 
          "면책허가결정":"면책허가결정공고\x7f", "신청취하" : "종국 : 신청취하\x7f", "이송" : "종국 : 이송\x7f", "인가" : r"(회생|변제)계획인가결정\x7f","인용" : "종국 : 인용\x7f",
          "폐지취소결정" :"폐지취소결정\x7f", "폐지취소결정송달" : "폐지취소결정 송달\x7f"}

기각폐지항고 = {"기각":"종국 : 기각\x7f", "기각결정":"개인회생절차개시신청 기각결정\x7f", 
          "폐지결정" : "폐지결정\(", "폐지":"종국 : 폐지\x7f", "항고" : "항고장"}

현재결과종류 = {key: value for d in [항고인용사건_현재결과종류, 기각폐지항고] for key, value in d.items()}

def 현재결과찾기(진행내용:str) -> str:
    # 진행내용 최근 결정내용을 찾아 현재결과 리턴
    for value in reversed(진행내용.split("\n")) :
        for i, v in 현재결과종류.items() :
            if re.search(v, value) :
                return i
            
    return "접수" # 처음까지 왔는데 아무런 결정내용도 확인할 수 없다면

def 항고사건_현재결과찾기(진행내용:str) -> str :
    for value in reversed(진행내용.split("\n")) :
        for i, v in 항고인용사건_현재결과종류.items() :
            if re.search(v, value) :
                return i
    return "접수" # 처음까지 왔는데 아무런 결정내용도 확인할 수 없다면

In [37]:
# previous = pd.read_excel(path_previous, sheet_name="개인회생(전체)", dtype={'채무자키':str, '분납키':str, '계좌키':str})
path_raw_data = join(path_dataset, name_dataset+".xlsx")
raw_data_ori = pd.read_excel(path_raw_data, dtype=rehabilitation_dtype).fillna("")

In [61]:
raw_data = raw_data_ori.copy()

##### 인가미인가

In [62]:
# 컬럼 순서 바꾸기, 인트를 문자열로(타입때메 종료제거보다 나중에 나와야)
raw_data.insert(1, "분납키", raw_data.pop("분납키"))
raw_data.insert(7, "현재결과", raw_data.pop("현재결과"))
raw_data.insert(8, "분납상태", raw_data.pop("분납상태"))
raw_data.insert(9, "검수상태", raw_data.pop("검수상태"))

# 현재결과 종류
# 인가 : 인가
# 미인가 : 개시, 금지, 금지명령
# 폐지 : 절차폐지결정일, 폐지결정, 기각, 기각결정, 신청취하, 폐지
# 면책 : 인용, 면책허가결정, 종결, 면책
# (230131) 개편
# 변제계획인가일 확인 할 것 : 폐지취소결정송달, 폐지취소결정
# 분납메모, 변제계획인가일 확인 할 것

# 인가미인가
인가미인가 = []
for i, v in tqdm(raw_data[["현재결과", "분납키", "분납메모", "변제계획인가일", "항고", "진행내용"]].iterrows(), total=len(raw_data.index)) :
    result, opb = "", 0
    
    결과 = v.현재결과
    # 현재결과에 따라 인가,미인가,폐지,면책,이송,체크 정하기
    # 인가
    if 결과 == "인가" : 
        result = "인가"
        
    # 미인가
    elif re.search("개시|금지|접수", 결과) : 
        result = "미인가"
        
    # 폐지
    elif (re.search("폐지결정|기각|취하", 결과)) or (결과 == "폐지") : 
        result = "폐지"
        
    # 폐지취소 - 분납메모 미사용
    elif re.search("폐지취소", 결과) : 
        result = "미인가" if v.변제계획인가일 == "" else "인가"
            
    # 면책
    elif re.search("인용|면책|종결", 결과) :
        result = "면책"
        
    # 이송 : 분납상태 중지 & 새사건 입력되었어야
    elif re.search("이송", 결과) :
        result = "이송"
        
    # 실사필요 : 항고인데 현재결과가 어디에도 해당하지 않을 때 or 새로운 현재결과가 나온 경우
    elif re.search("항고", 결과) : 
        항고사건결과 = v.분납메모.split(",")[0]
        채무자항고 = False if re.search("채권", v.항고) else True
        
        # 항고사건결과 아직 없음 : 체크
        if 항고사건결과 == "" : 
            result = "체크_항고사건결과없음"
            
        # 미결정 확인 완료 : 인가일 있는지에 따라
        elif 항고사건결과 == "미결정" : 
            result = "미인가" if v.변제계획인가일 == "" else "인가"
            
        # 항고 인용, 기각취소
        elif re.search("환송|인용|기각취소", 항고사건결과) :
            if 채무자항고 :
                result = "미인가" if v.변제계획인가일 == "" else "인가"
                raw_data.loc[i, "현재결과"] = 항고사건_현재결과찾기(v.진행내용)
            else : 
                result = "체크_채권자항고" # 채권자 항고에 대한 기각취소는 없어야하고, 파기환송과 인용은 아직 사례를 못 봐서 데이터 수집 필요
        # 취하, 기각, 각하
        elif re.search("취하|각하|기각(?!취소)", 항고사건결과) : 
            if 채무자항고 : 
                result = "폐지"
            else : 
                result = "미인가" if v.변제계획인가일 == "" else "인가"
                raw_data.loc[i, "현재결과"] = 항고사건_현재결과찾기(v.진행내용)
        # 분납메모에 엉뚱한 말이 적힌 경우
        else : 
            result = "체크_항고사건결과를 잘못적음"
    
    # 위의 어떤 경우에도 해당하지 않는 경우 : 없어야 함
    else : result = "체크_항목에 없는 현재결과"

    인가미인가.append(result)

raw_data.insert(7, "인가미인가", 인가미인가)
raw_data.insert(11, "반영OPB", None)

# 폐지취소 이전 파일에서 불러왔던 코드
# elif re.search("폐지취소", 결과) :  # 폐지취소결정송달 포함
#     temp = previous.query('분납키==@v.분납키')["인가미인가"].values
#     if len(temp) == 1 : result = temp[0]
#     else : result = "체크"

100%|██████████| 9494/9494 [00:00<00:00, 48601.51it/s]


##### 체크

In [63]:
# 진행건 중 체크 건 있는지 확인
수작업 = raw_data.query("인가미인가.str.contains('체크') and 분납상태 =='진행'")[["채무자키","계좌키","분납키","인가미인가","현재결과","분납메모"]]
if len(수작업) > 0: 
    display(수작업)
    수작업.to_excel("xxx.xlsx")
else : 
    print("진행건중 체크할 거 없음")

진행건중 체크할 거 없음


In [67]:
# check ----------------------------
cols = ["채무자키", "분납키", "분납자관계", "분납자성명인", "인가미인가", "현재결과", "분납상태", "중단사유", "분납중단일", "채무상태", "담당자", "개시결정일", "변제계획인가일"]
check = raw_data[cols].copy()
# 인가미인가 : 인가,미인가 // 폐지,면책,이송 // 체크
# 분납상태 : 진행, 중단

# 1. 인가미인가와 분납상태의 불일치 -> 인가 or 미인가 = 분납상태 진행
# 1-1.진행건 vs 상태 중단 : 중단사유 있는지?
중단사유적어 = check.query('인가미인가.str.contains("인가|미인가") & 분납상태=="중단" & 중단사유 == ""')
if len(중단사유적어) > 0 :
    print("중단사유 명시 필요") # 현재결과는 중단사유가 아니라 바로 확인 불가
    display(중단사유적어)
    
# 1-2. 중단건(인가/미인가 아닌것) vs 상태 진행                                            ################# 20106089는 일부러 중지 안함(240125)
진행건_인가미인가오류 = check.query('분납상태=="진행" & ~인가미인가.str.contains("인가|미인가") & 분납키 !="20106089"')
if len(진행건_인가미인가오류) :
    print("진행건인데 인가나 미인가가 아닌 것 있음")
    display(진행건_인가미인가오류)

# 2. not중지건 중 인가전 현재결과 vs 변제계획인가일 있는 경우
미인가건임 = check.query('분납상태=="진행" and 인가미인가=="미인가" and 변제계획인가일!=""')    ############## 20149032 는 회단사건으로 인가일 표기위치가 다름
인가건임 = check.query('분납상태=="진행" and 인가미인가=="인가" and 변제계획인가일=="" and 분납키!="20149032"')
if len(미인가건임) > 0 :
    print("미인가건인데 왜 인가일이 있음?")
    display(미인가건임)

if len(인가건임) > 0 :
    print("인가건인데 왜 인가일이 없음?")
    display(인가건임)

# 3. not중지건 중 개시결정일 있는데 전단계인 경우
# (반대)왜인지는 몰라도 개시결정일을 못 읽는 경우가 1~2건 있네.. 
개시건임 = check.query('분납상태=="진행" and 개시결정일 != "" and 현재결과.str.contains("금지|접수")')
개시건인데 = check.query('분납상태=="진행" and 개시결정일 == "" and 현재결과.str.contains("개시") and 분납키!="20149032"')
if len(개시건임) > 0 :
    print("개시일 있는데 왜 전단계?")
    display(개시건임)
    
if len(개시건인데) > 0 :
    print("개시인데 왜 개시결정일 없음? 20145963, 20149575 개시임은 확인했는데 결정일을 못 읽네..")
    display(개시건인데)

# 항고, 폐지취소 건 중 분납중지건은 확인 완료 (240130)
# 최신화 불가건 : 항고 뿐임 (폐지취소는 최신화 됨, 폐지 후 과거의 인가/미인가 여부를 확인하는 작업이 필요할뿐)
# 담당자, 채무상태, 누락 건 등은 자산정리 후 해야 정확

개시인데 왜 개시결정일 없음? 20145963, 20149575 개시임은 확인했는데 결정일을 못 읽네..


,채무자키,분납키,분납자관계,분납자성명인,인가미인가,현재결과,분납상태,중단사유,분납중단일,채무상태,담당자,개시결정일,변제계획인가일
3167,20413544,20145963,채무자,김유성,미인가,개시,진행,,,개인회생(진행중),개인회생,,
6137,20501359,20149575,채무자,배용수,미인가,개시,진행,,,개인회생(진행중),개인회생,,


In [ ]:
# 수정사항


In [68]:
# 일반내용 ~ 필터제출기준현재회차 내용삭제
ind1 = raw_data.columns.get_loc('일반내용')
ind2 = raw_data.columns.get_loc('필터제출기준현재회차')
raw_data.iloc[:,ind1:ind2] = ""

##### 통합신고,제외

In [50]:
# 통합신고, 제외
# 통합신고는 애초에 하나의 계좌로 금액을 합산하여 신고한 경우로 신고채권번호도 한 개. 
#       단일채무자키인 경우 전산 통합등록하고, 아무계좌에서나 변제처리 되어도 상관없다. - 통합등록한 경우 계좌 분리과정에서 대표계좌만 금액이 남는다.
#       다채무자키인 경우 통합등록은 불가, 따라서 대표가 아닌 계좌의 분납메모에는 반드시 통합신고 적어줘야 금액 0원처리 가능
# 제외는 계좌별로 신고하였으나, 원금이 적어 총분납금이 0원인 채권으로 신고채권번호는 여러개이나, 법원채권번호는 하나만 나온 경우
#       신규건은 분리등록이 원칙. 이미 통합등록이 된 경우 분리등록해줘야 하나 그대로 두고 제외가 아닌 통합신고로 처리해도 별 문제는 없음
####### 통합신고채권(채권번호 하나)은 계좌별로 등록할 필요가 없으니 이렇게 적을 필요가 없음(아래서, 계좌통합건은 행분열하고 모두 0원으로 처리, 현재원금은 유지)
####### 통합신고채권은 계좌별로 분리하지 말고 통합해서 전산등록해야
raw_data.loc[raw_data.분납메모.str.contains("통합|제외"), ["총분납금","분납미납금"]] = 0
raw_data.loc[raw_data.분납메모.str.contains("통합|제외"), ["분납잔금","총분납잔금"]] = raw_data['총분납금'] - raw_data["총분납입금"]


# # 엑셀 파일로 저장
# if not os.path.exists(path_result):
#     with pd.ExcelWriter(path_result, mode='w', engine='openpyxl') as writer:
#         raw_data.to_excel(writer, sheet_name='개인회생(전체)', index=False)
# else : 
#     with pd.ExcelWriter(path_result, engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer :
#         raw_data.to_excel(writer, sheet_name='개인회생(전체)', index=False)

In [29]:
# 공채권 확인
raw_data.loc[raw_data.분납메모.str.match("(공|통합)채권"),:]

,채무자키,분납키,채권구분,매각사구분,분납자관계,분납자성명인,계좌키,인가미인가,현재결과,분납상태,...,최종진행결과,사건키,자동최신화,최신화결과화면,관련최신화결과화면,공고게시일,공고물,공고변제율,공고변제기간,NOSER
2136,20410192,20105691,IRL(인가),고려저축-01,채무자,박승만,200974824,면책,인용,중단,...,,20378945,,Y,,,,0,0,
3231,20421340,20146420,NPL,한울가람,채무자,금기정,200938541,인가,인가,진행,...,,2301027475,D,Y,,,,0,0,
3673,20430551,20119818,IRL,유안타저-01,채무자,송창현,200951164,면책,인용,중단,...,,20418573,,Y,,,,0,0,
3679,20430557,20119822,IRL,유안타저-01,채무자,이상준,200951209,면책,인용,중단,...,,20418577,,Y,,,,0,0,
3985,20436703,20125480,NPL,KB카드-01,채무자,오복희,200957789,인가,인가,진행,...,,20455053,D,Y,,,,0,0,
4594,20451606,20119907,IRL(인가),고려저축-01,채무자,정현철,200974148,면책,인용,중단,...,,20418662,,Y,,,,0,0,
4644,20451815,20125237,CCRS(개인),고려저축-01,채무자,조영성,200974471,인가,인가,진행,...,,20453349,D,Y,,,,0,0,
4679,20451911,20119946,IRL(미인가),고려저축-01,채무자,지수홍,200974616,인가,인가,진행,...,,20418701,D,Y,,,,0,0,
4689,20451954,20119949,IRL(인가),고려저축-01,채무자,전순여,200974678,인가,인가,진행,...,,20418704,D,Y,,,,0,0,
4708,20451989,20119959,IRL(인가),고려저축-01,채무자,윤성중,200974745,인가,인가,진행,...,,20418714,D,Y,,,,0,0,


In [68]:
#예전 수작업 내용 전산조회 할 거 작업 후
# - 분납상태-[진행] & 
# - 현재결과-[폐지취소결정, 폐지취소결정송달, 빈셀] & 
# - 인가미인가-[기타, #N/A, ""] 즉 인가,미인가가 아닌 거
# - 매각 제외 전이므로, 채무상태 및 담당자에서 종결, 매각건 제외

##### OPB 및 연납건 처리

In [60]:
raw_data.query('인가미인가.str.contains("인가|미인가|항고")').분납상태.value_counts()

분납상태
진행    5266
중단     741
Name: count, dtype: int64

In [ ]:
# 연납건 처리-----------------------------------------------------------------------------------------------------------------------------------
# 20420209 정순만 통합등록되어있기 때문에 계좌키 분리하고 처리하려고 했는데 기존 입금이 지맘대로 잡혀있어서 그냥 여기서 처리하고 2번계좌는 계좌분리시 0원으로 되게끔
연납건 = raw_data.query('분납키 == "20149032"').index
총분납회차 = 10
############## 1년에 한번씩만 수정하면 됨
현재회차 = 2
납입회차 = 2
###############
잔여회차 = 총분납회차 - 현재회차
미납회차 = 현재회차 - 납입회차
변제금 = 2972696

raw_data.loc[연납건, "총분납회차"], raw_data.loc[연납건, "현재회차"], raw_data.loc[연납건, "납입회차"], raw_data.loc[연납건, "잔여회차"], raw_data.loc[연납건, "미납회차"] = 총분납회차, 현재회차, 납입회차, 잔여회차, 미납회차
raw_data.loc[연납건, "총분납금"],raw_data.loc[연납건, "총분납입금"], raw_data.loc[연납건, "분납미납금"], raw_data.loc[연납건, "분납잔금"], raw_data.loc[연납건, "총분납잔금"] = 총분납회차*변제금, 납입회차*변제금, 미납회차*변제금, 잔여회차*변제금, (총분납회차-납입회차)*변제금
# 연납건 처리 끝-----------------------------------------------------------------------------------------------------------------------------------

# OPB
raw_data['반영OPB'] = np.where(raw_data['인가미인가'] == '인가', raw_data["총분납잔금"] , raw_data["현재원금"])


##### 진행건 분리

In [73]:
# 진행중, 차주, 보증인 시트 작성(종결인데 분납중지 안 된 것 여기선 제외x)
ing_df = raw_data.query('분납상태 == "진행" and 인가미인가.str.contains("인가|미인가")')

# 종료건 제외 : 일부 계좌만 종결|매각|환매 될 수 있으며, 보증인이 회생 중일 수 있으므로 여기서 제외하지 않는다.(어차피 통합에선 종결건은 우선 제외하므로 자산정리에 문제는 없음)
# 통합파일에서 계좌별로 상태 확인 후 종료인데 중지 안 된 건 처리
# if path_dataset.find("대성") < 0 : # 솔림
#     ing_df = raw_data[~(raw_data.채무상태.str.match("종결|환매|매각")|raw_data.담당자.str.match("종결|매각(?!예정)|DS|환매"))]
# else : # 대성
#     ing_df = raw_data[~(raw_data.채무상태.str.match("종결|환매|매각")|raw_data.담당자.str.match("종결|매각|환매|파산|회계파산"))]

    
###### 신복과 충돌하는 거 제거하기 #############################################################
if path_dataset.find("대성") < 0 : # 솔림
    개회제외 = {"채무자" : [], "보증인" : [] } # 20462830 중복입금건. 신복으로 처리하다가 개인회생 면책됨 231106
else : # 대성
    개회제외 = {"채무자" : [], "보증인" : [] } # 
##############################################################################################
if len(개회제외["채무자"]) > 0 : ing_df = ing_df[~(ing_df['채무자키'].isin(개회제외["채무자"]) & (ing_df['분납자관계'] == '채무자'))]
if len(개회제외["보증인"]) > 0 : ing_df = ing_df[~(ing_df['채무자키'].isin(개회제외["보증인"]) & (ing_df['분납자관계'] == '보증인'))]

# 채무자, 보증인
debt_ing = ing_df[ing_df["분납자관계"]=="채무자"]
grt_ing = ing_df[ing_df["분납자관계"]=="보증인"]

# 저장
with pd.ExcelWriter(join(path_dataset, name_dataset+"_기준데이터.xlsx"), engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer :
    ing_df.to_excel(writer, sheet_name='개인회생(진행)', index=False)
    debt_ing.to_excel(writer, sheet_name='개인회생(진행)_채무자', index=False)
    grt_ing.to_excel(writer, sheet_name='개인회생(진행)_보증인', index=False)
    raw_data.to_excel(writer, sheet_name='개인회생(전체)', index=False)

##### 계좌키 나누고 새로운 파일 저장
- 통합계좌 없으면 대성은 안 해도 됨

In [74]:
# 개인회생 계좌 나누기

# path_dataset = r"D:\3.자산\전산 dataset\230430(0501)"
# name_dataset = "개인회생새창_20230502_0850_기준데이터"
# ing_df = pd.read_excel(join(path_dataset, name_dataset+".xlsx"), sheet_name="개인회생(진행)", dtype={'채무자키':str, '분납키':str, '계좌키':str})

split_ing_df = ing_df.copy()
split_ing_df["비고"] = ""
new_rows = []
for index, row in split_ing_df.iterrows():
    account_keys = row['계좌키'].split(',')
    first_account_key = account_keys.pop(0)
    split_ing_df.loc[index, '계좌키'] = first_account_key # 여러 계좌키를 첫번째 계좌키로 값 수정

    for account_key in account_keys:
        new_row = row.copy() # 일단 모든 칼럼 복사
        new_row['계좌키'] = account_key
                            # 전산 OPB는 최초OPB임
        new_row['반영OPB'],new_row['OPB'], new_row['총분납금'],new_row['총분납입금'],new_row['분납미납금'],new_row['분납잔금'],new_row['총분납잔금'] = 0,0,0,0,0,0,0 # 값 안 가져올 컬림은 빈문자열로 대체
        new_row["분납메모"] = "통합신고 " + first_account_key
        new_rows.append(new_row)
        

temp = pd.DataFrame(new_rows)

result = pd.concat([split_ing_df, temp], ignore_index=True)

# 저장
writer = pd.ExcelWriter(join(path_dataset, name_dataset+"_진행중_계좌키분리_기준데이터.xlsx"))
result.to_excel(writer, sheet_name="개인회생(진행)", index=False)
# writer.save() # 이제 save()는 없어진 api 안 쓰면 됨.
writer.close()
